![LangChain](img/langchain.jpeg)

Les systèmes **RAG (Retrieval-Augmented Generation)** dans LangChain permettent aux modèles de langage de s’appuyer sur des **connaissances externes** pour produire des réponses plus précises, actualisées et pertinentes.

Contrairement à un simple LLM qui génère une réponse uniquement à partir de ce qu’il a appris pendant son entraînement, un système RAG interroge une base de documents pour retrouver des morceaux d’information pertinents – appelés **chunks** – et les injecte dans le prompt du LLM.

![RAG](img/rag.jpeg)

**Que montre le schéma ci-dessus ?**

Le processus se divise en **deux grandes phases** : **préparation des documents** et **traitement des requêtes**.

**Préparation des documents (à gauche)**
- (1) Un fichier (document source) est divisé en **chunks**, c’est-à-dire en petits segments de texte.
- (2) Chaque chunk est passé dans un LLM Embedder, un encodeur qui transforme le texte en un vecteur numérique (**embeddings**).
- (3) Ces vecteurs sont ensuite stockés dans un Vector Store, une base de données spécialisée pour les recherches par **similarité sémantique**.

**Traitement des requêtes (à droite)**
- (a) Lorsqu’un utilisateur emet une requête, celle-ci est à son tour encodée via **le même LLM Embedder** pour obtenir son vecteur.
- (b) Ce vecteur est utilisé par le **Retriever**, qui compare la requête aux vecteurs des **chunks** pour trouver les plus similaires.
- (c) Les chunks retrouvés sont envoyés au LLM, qui les utilise comme contexte pour formuler une réponse.


En résumé, ce fonctionnement est illustré par la boucle :

> Requête → Encodage → Recherche dans la base vectorielle → Récupération des chunks → Passage au LLM → Réponse contextuelle


# 1. Chargement du modèle LLM local
___

Dans cette section, nous chargeons un modèle de langage local grâce à **Ollama**. Cela permet de travailler avec un **LLM directement sur notre machine**, sans connexion à une API externe.

Nous utilisons ici la classe `ChatOllama` de **LangChain**, qui nous permet d’interagir facilement avec un modèle comme **llama3** ainsi qu'un **modèle d'embeddings** déjà téléchargés via Ollama.

In [2]:
import os
from IPython.display import display, Markdown, clear_output
from dotenv import load_dotenv
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain

# Chargement des clés d'API se trouvant dans le fichier .env.  
# Ceci permet d'utiliser des modèles en ligne comme gpt-x, deepseek-x, etc...
load_dotenv(override=True)

model = ChatOllama(model="llama3", temperature=0)
#model = ChatDeepSeek(model="deepseek-chat", api_key=os.getenv("DEEPSEEK_API_KEY"))

# Modèle spécialisé pour convertir du texte en vecteurs (https://ollama.com/library/nomic-embed-text).
# Il existe d'autres modèles d'embeddings (comme "all-MiniLM-L6-v2", "text-embedding-ada-002", etc.) 
# avec des performances et dimensions variées selon les cas d’usage (recherche sémantique, classification, etc.).
embedder = OllamaEmbeddings(model="nomic-embed-text")


# 2. RAG standard
___

Le **RAG standard** consiste à :
- formuler une requête explicite
- interroger une base de documents vectorisée
- utiliser un modèle LLM pour générer une réponse à partir des résultats retrouvés. 
 
Ce pipeline est **efficace pour des questions indépendantes, sans contexte conversationnel**.

### 2.1 Préparation des documents

Nous initialisons les chemins nécessaires à la préparation des documents d’entrée.

In [3]:
# Récupère le chemin absolu du répertoire courant (là où le script est exécuté)
current_dir = os.getcwd()

# Nom du fichier texte contenant les comptes rendus de réunion
file_name = "meeting_reports.txt"

# Construit le chemin complet vers le fichier texte dans le dossier "data"
file_path = os.path.join(current_dir, "data", file_name)

# Définit le chemin du répertoire où sera stockée la base de données vectorielle (Chroma DB)
db_dir = os.path.join(current_dir, "data", "db")

### 2.2 Initialisation du vector store

Nous vérifions ici si la base vectorielle existe déjà.  
Si ce n’est pas le cas, le fichier source est chargé, découpé en morceaux, enrichi de métadonnées, puis indexé dans Chroma DB.

In [4]:
if not os.path.exists(db_dir):
    print("Initializing vector store...")

    # Chargement du fichier texte brut contenant les documents
    loader = TextLoader(file_path)
    loaded_document = loader.load()

    # Découpage du document en chunks de 1000 caractères avec un chevauchement de 0
    # - chunk_size détermine la taille maximale de chaque morceau (en nombre de caractères ici : 1000)
    # - chunk_overlap permet de conserver un chevauchement entre les morceaux pour éviter les coupures abruptes, ici il est à 0, donc sans recouvrement.
    # - RecursiveCharacterTextSplitter est souvent préféré en pratique pour des documents textuels comme des comptes rendus, 
    #   des articles ou de la documentation technique, car il garde mieux le contexte sémantique.
    #   Ce splitter tente d'abord de découper sur les sauts de ligne, puis sur les phrases, puis sur les mots, etc.
    # ... d'autres Text Splitter comme CharachterTextSplitter existent. À approfondir si besoin
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    chunks = text_splitter.split_documents(loaded_document)

    # Ajout de métadonnées à chaque chunk (utile pour le filtrage ou le suivi de provenance).
    # Ici 2 metadata sont ajoutés mais il pourrait en y avoir plus.
    for chunk in chunks:
        chunk.metadata["source"] = file_path    # Chemin d'origine du document
        chunk.metadata["category"] = "meeting"  # Catégorie de contenu (à adapter selon les besoins)

    # Création et persistance de la base vectorielle dans le dossier défini
    db = Chroma.from_documents(chunks, embedder, persist_directory=db_dir)

    print("Vector store created !")

### 2.3 Initialisation du moteur de recherche vectorielle

Une fois la base vectorielle Chroma initialisée avec les embeddings, nous la transformons en **moteur de recherche (retriever)**.  
Cela permet de retrouver les documents les plus proches sémantiquement d’une question ou d’une requête.  

In [5]:
# Chargement de la base vectorielle existante, avec liaison avec le même embedder ayant servi pour créer la base vectorielle
db = Chroma(persist_directory=db_dir, embedding_function=embedder)

# Conversion de la base Chroma en "retriever" pour effectuer des recherches par similarité
# - search_type="similarity" utilise la distance cosinus entre les vecteurs
# - "k": 3 signifie que l'on souhaite récupérer les 3 documents les plus proches
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# 💡 Il est aussi possible d’utiliser d’autres types de recherche (search_type) :
# - "mmr" (Maximal Marginal Relevance) : équilibre entre pertinence et diversité des résultats
# - "similarity_score_threshold" : retourne uniquement les documents dont le score dépasse un certain seuil
#      search_kwargs={"score_threshold": 0.8} permet par exemple de filtrer les résultats peu pertinents
#
# D’autres paramètres utiles dans search_kwargs :
# - "fetch_k" : nombre de documents à récupérer avant le tri final (utile avec MMR)
# - "lambda_mult" : pondération entre pertinence et diversité dans MMR
# 
# Etc... à approfondir si besoin

/var/folders/59/q0j81gmx5h7gvpg38k8hknzw0000gn/T/ipykernel_1472/573588974.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=db_dir, embedding_function=embedder)


### 2.4 Exécution d’une requête de recherche

Dans cette étape, nous combinons la recherche vectorielle avec un LLM.  
L’objectif est de fournir une réponse pertinente à une question, en s’appuyant uniquement sur les documents retrouvés dans la base vectorielle.  
Le modèle est guidé par un prompt structuré qui inclut la requête initiale et les contenus des chunks pertinents.

In [6]:
# Requête posée par l'utilisateur
query = "Quels sont les réunions concernant la société Neolink ?"

# Recherche des chunks vectoriellement proches de la question
relevant_chunks = retriever.invoke(query)

# Optionnel : affichage manuel des chunks retrouvés (utile pour debug ou vérification)
# for i, chunk in enumerate(relevant_chunks, 1):
#     print(f"Chunk {i}:\n{chunk.page_content}\n")

# Construction du message d'entrée à envoyer au modèle
# Nous incluons la question et le contenu des documents pour contraindre le LLM à ne répondre qu'en s'appuyant sur ces sources
input_message = (
    "Voici des documents qui vont t'aider à répondre à la question : "
    + query
    + "\n\nDocuments pertinents : \n"
    + "\n\n".join([chunk.page_content for chunk in relevant_chunks])
    + "\n\nDonne une réponse basée uniquement sur les documents qui te sont fournis."
)

# Construction du message complet pour le LLM, avec un rôle système et un message utilisateur
messages = [
    SystemMessage(content="Tu es un assistant qui aide à retrouver tout type d'informations interne à une entreprise"),
    HumanMessage(content=input_message)
]

result = model.invoke(messages)

display(Markdown(result.content))

En examinant les documents que vous m'avez fournis, je peux voir que les réunions concernant la société Neolink sont :

* La réunion du 09/03/2025 (Bilan des incidents techniques du mois) : Cette réunion a porté sur les tâches réalisées et les éléments bloquants liés aux interactions avec NeoLink.
* La réunion du 10/05/2025 (Réunion hebdomadaire de coordination technique) : Cette réunion a également abordé les enjeux liés à la communication interne, jugée parfois insuffisante, et aux besoins en ressources.

Il n'y a pas d'autres réunions mentionnées dans les documents que vous m'avez fournis.

### 🧩 Exercice

La société NovTech gère de nombreux documents internes :
- des rapports d’incidents (panne, erreur technique, post-mortem),
- des procédures opérationnelles (onboarding, accès système, déploiement…).

Actuellement, les équipes perdent du temps à chercher les bonnes informations à travers des fichiers éparpillés.

Votre objectif est de construire un assistant basé sur l'architecture RAG qui permettra :
- de retrouver rapidement les procédures en cas de besoin,
- de consulter les résolutions d’incidents similaires,
- de répondre à des questions en langage naturel en s’appuyant uniquement sur les documents internes.

Pour vous aider, vous pouvez suivre les étapes suivantes :
1. Chargement des documents
2. Découpage en chunks
3. Indexation vectorielle
4. Recherche contextuelle
5. Génération de réponse

ℹ️ Les documents de l'entreprise se trouve dans le dossier `data/novtech`.  
💪🏻 **Bonus** : Rendre possible un filtrage par catégorie dans les recherches

In [7]:
# Votre code ici

# 3. RAG conversationnel
___

Dans un cadre d’**interaction continue**, les utilisateurs posent souvent des questions implicites ou référentielles (ex. “Et lui ?”). Le **RAG conversationnel** ajoute une étape clé : la **reformulation de la question en prenant en compte l’historique du dialogue**.  

Cette version de RAG permet de maintenir la pertinence des recherches dans la base vectorielle tout en conservant la fluidité de la conversation, ce qui la rend adaptée aux assistants IA ou aux chatbots avancés.

**Exemple**

Historique de la conversation :
- Utilisateur : *Qui est le CEO de Tesla ?*
- IA : *Elon Musk est le CEO de Tesla*.
- Utilisateur : *Et de SpaceX ?*

➡️ La question “Et de SpaceX ?” est ambiguë seule. Le moteur de recherche (retriever) ne sait pas de quoi il s’agit exactement.

Avec une reformulation de la question de l'utilisateur cela donnerait : “Qui est le CEO de SpaceX ?”

➡️ Résultat : la requête est claire, et la recherche dans la base vectorielle peut retourner les bons documents.

**👍 LangChain facilite ce processus**

LangChain fournit une abstraction prête à l’emploi grâce à la classe `ConversationalRetrievalChain`.
Cette classe prend automatiquement en charge :
- la reformulation de la question via le LLM
- la recherche dans la base vectorielle
- la génération de la réponse finale à partir des documents récupérés et de l’historique

➡️ Elle encapsule ainsi toute la logique conversationnelle d’un RAG en une seule ligne.

In [8]:
# Chaîne RAG avec historique
qa_chain = ConversationalRetrievalChain.from_llm(llm=model, retriever=db.as_retriever())

# Boucle de chat
chat_history = []
while True:
    user_input = input("You: ")
    clear_output(wait=True)                         # Efface l'affichage précédent
    display(Markdown(f"**Vous :** {user_input}"))   # Affiche la requête de l'utilisateur

    if user_input.lower() in ["stop", "exit", "quit"]:
        print("Fin de la conversation.")
        break

    result = qa_chain({"question": user_input, "chat_history": chat_history})
    display(Markdown(result["answer"]))
    chat_history.append((user_input, result["answer"]))

**Vous :** quit

Fin de la conversation.


### 🧩 Exercice

Repartez de l'exercice précédent (NovTech), et implémentez un assistant de conversation continue.

In [ ]:
# Votre code ici